In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.imports import *
from lib.video_processor import VideoProcessor
from lib.misc import *

/home/tobi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# sudo modprobe v4l2loopback
# sudo modprobe v4l2loopback devices=1 card_label="Virtual Camera" exclusive_caps=1
# sudo modprobe -r v4l2loopback && sudo modprobe v4l2loopback devices=1 video_nr=4 card_label="Virtual" exclusive_caps=1 max_buffers=2

In [ ]:
interpolate_rate=2
eyes_int_rate=0
eye_model_size=128
input_source = "example_video/youtube_vid_04.mp4" # 0 # "example_video/example_vid_original.mp4"
form="onnx"


eye_batch_rate=(interpolate_rate//(eyes_int_rate+1))+1
convert_pt_to_onnx(model_path=f'models/eyes_{eye_model_size}_auto.pt', imgsz=eye_model_size, batch=eye_batch_rate, form=form)

processor = VideoProcessor(eye_model_size=eye_model_size, form=form)
out = processor.start(video_path=input_source, 
                      interpolate_rate=interpolate_rate, 
                      eyes_int_rate=eyes_int_rate, 
                      eye_batch_rate=eye_batch_rate, 
                      return_instead=False, 
                      disable_stats=False, 
                      save_output=True, 
                      file_process_fast=False,
                      make_virtual_cam=True);

Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8n-pose summary (fused): 187 layers, 3154073 parameters, 0 gradients, 8.6 GFLOPs

PyTorch: starting from 'models/eyes_128_auto.pt' with input shape (3, 3, 128, 128) BCHW and output shape(s) (3, 35, 336) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.7s, saved as 'models/eyes_128_auto.onnx' (12.1 MB)

Export complete (2.1s)
Results saved to /home/tobi/Desktop/eye_tracking/models
Predict:         yolo predict task=pose model=models/eyes_128_auto.onnx imgsz=128  
Validate:        yolo val task=pose model=models/eyes_128_auto.onnx imgsz=128 data=data/yolo_pose_dataset.yaml  
Visualize:       https://netron.app
Loading models/yolov8n-face-lindevs.onnx for ONNX Runtime inference...
Loading models/eyes_128_auto.onnx for ONNX Runtime i

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


writing video of shape (1798, 720, 1280, 3) to path outputs/youtube_vid_04.mp4 with frame_rate 29.97002997002997, should be 59.99326666666667 seconds long


 26%|██████████▎                            | 475/1798 [00:02<00:07, 181.70it/s]

In [ ]:
# write_video(out, "example_video/flexpressions_cropped.mp4")

## Training

In [ ]:
prepare_video_annotations_for_training(val_ratio=0.1, multiply_train_data=32)

In [ ]:
# if you don't want to track training with comet don't run this cell and set usecomet=False in the next cell
init_comet(api_key="your_api_key")

In [ ]:
for image_size in [128, 256, 64, 512]: # might get stuck before starting 2nd batch, if this happens just restart this cell
    train_model(pretrained_model='models/yolov8n-pose.pt', save_name=f"training_{image_size}_autotrain", imgsz=image_size, epochs=100, batch=256, amp=True, workspace="comet_workspacename", usecomet=False)